In [2]:
import pandas as pd

# Replace 'yourfile.csv' with the path to your CSV file
file_path = R"C:\Users\svars\OneDrive\Desktop\Venkat\archive (3)\Resume\Resume.csv"
data = pd.read_csv(file_path)




In [3]:
data.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [4]:
import requests
from io import StringIO

# URL pointing to the JSON Lines file
url = 'https://raw.githubusercontent.com/mystery2life/NLP-Project/main/temp_patterns.jsonl'

# Fetching the data from the URL
response = requests.get(url)
response.raise_for_status()  # This will raise an exception if there's an error

# Converting JSON Lines string to a pandas DataFrame
SkillData = pd.read_json(StringIO(response.text), lines=True)

In [5]:

pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [7]:
!python -m spacy download en_core_web_lg

     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     -------------------------------------- 0.1/587.7 MB 871.5 kB/s eta 0:11:15
     ---------------------------------------- 0.2/587.7 MB 1.7 MB/s eta 0:05:38
     ---------------------------------------- 0.5/587.7 MB 3.1 MB/s eta 0:03:09
     ---------------------------------------- 0.8/587.7 MB 4.2 MB/s eta 0:02:19
     ---------------------------------------- 1.2/587.7 MB 4.9 MB/s eta 0:01:59
     ---------------------------------------- 1.5/587.7 MB 5.5 MB/s eta 0:01:48
     ---------------------------------------- 1.9/587.7 MB 5.7 MB/s eta 0:01:42
     ---------------------------------------- 2.4/587.7 MB 6.3 MB/s eta 0:01:33
     ---------------------------------------- 2.8/587.7 MB 6.6 MB/s eta 0:01:29
     ---------------------------------------- 3.2/587.7 MB 6.8 MB/s eta 0:01:27
     ---------------------------------------- 3.7/587.

In [6]:
import spacy

# Load the pre-trained spaCy model
nlp = spacy.load("en_core_web_lg")

In [7]:
from spacy.matcher import Matcher

In [8]:
matcher = Matcher(nlp.vocab)

# Add patterns directly without using eval
for index, row in SkillData.iterrows():
    pattern = row['pattern']  # Assuming this is already a list of dictionaries
    matcher.add("SKILL", [pattern])

In [9]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [10]:
from tqdm import tqdm

def extract_skills(resume_texts):
    training_data = []
    for resume in tqdm(resume_texts, desc="Extracting skills"):
        doc = nlp(resume)
        matches = matcher(doc)
        spans = [doc[start:end] for _, start, end in matches]
        entities = [(span.start_char, span.end_char, "SKILL") for span in spans if span.text.strip()]
        training_data.append((resume, {"entities": entities}))
    return training_data


In [11]:
# Limit the data to the first 5 resumes
resume_texts_sample = data['Resume_str'].tolist()

# Extract skills from the limited sample
training_data_sample = extract_skills(resume_texts_sample)  

Extracting skills: 100%|██████████| 2484/2484 [10:28<00:00,  3.95it/s]


In [12]:
print(training_data_sample[1])

("         HR SPECIALIST, US HR OPERATIONS       Summary     Versatile  media professional with background in Communications, Marketing, Human Resources and Technology.\xa0        Experience     09/2015   to   Current     HR Specialist, US HR Operations    Company Name   －   City  ,   State       Managed communication regarding launch of Operations group, policy changes and system outages      Designed standard work and job aids to create comprehensive training program for new employees and contractors         Audited job postings for old, pending, on-hold and draft positions.           Audited union hourly, non-union hourly and salary background checks and drug screens             Conducted monthly new hire benefits briefing to new employees across all business units               Served as a link between HR Managers and vendors by handling questions and resolving system-related issues         Provide real-time process improvement feedback on key metrics and initiatives  Successfully 

In [13]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [14]:
from sklearn.model_selection import train_test_split

# Assuming 'training_data_sample' is a list of tuples in the format (text, annotations)
texts, annotations = zip(*training_data_sample)  # Unzip the training data

# Split the data into train and validation sets
train_texts, val_texts, train_annotations, val_annotations = train_test_split(
    texts, annotations, test_size=0.2, random_state=42  # 20% of the data is used for validation
)

# Recreate the training and validation sets as tuples (text, annotation)
train_data = list(zip(train_texts, train_annotations))
val_data = list(zip(val_texts, val_annotations))


In [15]:
import random

In [28]:
import spacy
import random
from tqdm import tqdm
from spacy.training import Example



# Get the NER pipeline component
ner = nlp.get_pipe("ner")

# Function to remove overlapping entities
def remove_overlapping_entities(entities):
    entities = sorted(entities, key=lambda x: x[0])  # Sort by start position
    non_overlapping_entities = []
    last_end = -1
    for start, end, label in entities:
        if start >= last_end:  # No overlap
            non_overlapping_entities.append((start, end, label))
            last_end = end
    return non_overlapping_entities

# Add the 'SKILL' entity label to the NER model if it's not already known
if 'SKILL' not in ner.labels:
    ner.add_label("SKILL")

# Additional custom entities from training data
for _, annotations in train_data:
    for ent in annotations.get("entities"):
        if ent[2] not in ner.labels:
            ner.add_label(ent[2])

# Disable other pipeline components during training to train only NER
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']




In [29]:
with nlp.disable_pipes(*unaffected_pipes):  # only train NER
    optimizer = nlp.resume_training()
    for itn in range(1):  # Number of training iterations
        random.shuffle(train_data)
        losses = {}
        for text, annotations in tqdm(train_data, desc=f"Iteration {itn}"):
            entities = remove_overlapping_entities(annotations.get("entities"))
            annotations['entities'] = entities  # Update annotations with non-overlapping entities
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, {'entities': entities})
            nlp.update([example], drop=0.2, sgd=optimizer, losses=losses)
        print(f"Iteration {itn}, Losses: {losses}")

Iteration 0: 100%|██████████| 1987/1987 [09:11<00:00,  3.60it/s]

Iteration 0, Losses: {'ner': 3501.3881677342483}


In [30]:
import os

# Define a valid path for saving the model
model_dir = R"C:\Users\svars\OneDrive\Desktop\Venkat\final\NLP-Project-main\ModelsV2\saved_modelv2"

# Create the directory if it does not exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the trained model to disk
nlp.to_disk(model_dir)

# Load the model from the saved directory into a new variable
newnlp = spacy.load(model_dir)


In [35]:
import spacy
from spacy.tokens import Span
from spacy.training import Example
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    examples_to_score = []
    
    for input_text, annot in examples:
        # Create doc for gold text
        doc_gold_text = ner_model.make_doc(input_text)
        spans = [Span(doc_gold_text, start, end, label) for start, end, label in annot['entities'] if start >= 0 and end <= len(doc_gold_text)]
        
        # Set gold spans to the .ents attribute directly for correct scoring
        doc_gold_text.ents = spans

        # Process the text with the NER model to get predictions
        pred_doc = ner_model(input_text)
        
        # Create an Example object with the predicted and gold docs
        example = Example(pred_doc, doc_gold_text)
        examples_to_score.append(example)

    # Score the examples and calculate NER metrics
    scores = scorer.score(examples_to_score)

    # Return only NER-related metrics
    return {
        'ents_p': scores['ents_p'],
        'ents_r': scores['ents_r'],
        'ents_f': scores['ents_f'],
        'ents_per_type': scores['ents_per_type']
    }


In [41]:
test = "Abid Ali Awan Data Scientist I am a certified data scientist professional, who loves building machine learning models and blogs about the latest AI technologies. I am currently testing AI Products at PEC-PITC, which later gets approved for human trials. abidaliawan@tutamail.com +923456855126 Islamabad, Pakistan abidaliawan.me WORK EXPERIENCE Data Scientist Pakistan Innovation and Testing Center - PEC 04/2021 - Present, Islamabad, Pakistan Redesigned data of engineers that were mostly scattered and unavailable. Designed dashboard and data analysis report to help higher management make better decisions. Accessibility of key information has created a new culture of making data-driven decisions. Contact: Ali Raza Asif - darkslayerraza10@gmail.com Data Scientist Freelancing/Kaggle 11/2020 - Present, Islamabad, Pakistan Engineered a healthcare system. Used machine learning to detect some of the common decisions. The project has paved the way for others to use new techniques to get better results. Participated in Kaggle machine learning competitions. Learned new techniques to get a better score and finally got to 1 percent rank. Researcher / Event Organizer CREDIT 02/2017 - 07/2017, Kuala Lumpur, Malaysia Marketing for newly build research lab. Organized technical events and successfully invited the multiple company's CEO for talks. Reduced the gap between industries and educational institutes. Research on new development in the IoT sector. Created research proposal for funding. Investigated the new communication protocol for IoT devices. Contact: Dr. Tan Chye Cheah - dr.chyecheah.t@apu.edu.my EDUCATION MSc in Technology Management Staffordshire University 11/2015 - 04/2017, Postgraduate with Distinction Challenges in Implementing IoT-enabled Smart cities in Malaysia. Bachelors Electrical Telecommunication Engineering COMSATS Institute of Information Technology, Islamabad 08/2010 - 01/2014, CGPA: 3.09 Networking Satellite communications Programming/ Matlab Telecommunication Engineering SKILLS Designing Leadership Media/Marketing R/Python SQL Tableau NLP Data Analysis Machine learning Deep learning Webapp/Cloud Feature Engineering Ensembling Time Series Technology Management ACHIEVEMENTS 98th Hungry Geese Simulation Competition (08/2021) 2nd in Covid-19 vaccinations around the world (07/2021) 8th in Automatic Speech Recognition in WOLOF (06/2021) Top 10 in WiDS Datathon. (03/2021) 40th / 622 in MagNet: Model the Geomagnetic Field Hosted by NOAA (02/2021) 18th in Rock, Paper, Scissors/Designing AI Agent Competition. (02/2021) PROJECTS Goodreads Profile Analysis WebApp (09/2021) Data Analysis Web Scraping XLM Interactive Visualization Contributed in orchest.io (08/2021) Testing and Debuging Technical Article Proposing new was to Improve ML pipelines World Vaccine Update System (06/2021) Used sqlite3 for database Automated system for daily update the Kaggle DB and Analysis Interactive dashboard mRNA-Vaccine-Degradation-Prediction (06/2021) Explore our dataset and then preprocessed sequence, structure, and predicted loop type features Train deep learning GRU model Trip Advisor Data Analysis/ML (04/2021) Preprocessing Data, Exploratory Data analysis, Word clouds. Feature Engineering, Text processing. BiLSTM Model for predicting rating, evaluation, model performance. Jane Street Market Prediction (03/2021) EDA, Feature Engineering, experimenting with hyperparameters. Ensembling: Resnet, NN Embeddings, TF Simple NN model. Using simple MLP pytorch model. Achievements/Tasks Achievements/Tasks Achievements/Tasks Thesis Courses"

In [42]:
# Assuming 'newnlp' is your spaCy NER model already loaded
doc = newnlp(test)  # Process the text through the model

# Print the recognized entities and their labels
for ent in doc.ents:
    print(ent.text, ent.label_)

AI SKILL
testing SKILL
AI SKILL
Testing SKILL
data analysis SKILL
Marketing SKILL
Engineering SKILL
communications SKILL
Engineering SKILL
Marketing SKILL
R SKILL
Python SKILL
SQL SKILL
Tableau SKILL
Data Analysis SKILL
Machine learning SKILL
Webapp SKILL
Engineering SKILL
Simulation SKILL
AI SKILL
WebApp SKILL
Data Analysis SKILL
Visualization SKILL
Testing SKILL
ML SKILL
database SKILL
Data Analysis SKILL
ML SKILL
Data analysis SKILL
Engineering SKILL
Engineering SKILL
